In [ ]:
"""下载LLaMA-Factory
"""
!git clone https://github.com/hiyouga/LLaMA-Factory

In [ ]:
"""安装依赖
"""
%cd /content/LLaMA-Factory
!pip install -r requirements.txt
!pip install einops
!pip install transformers==4.34.0
!pip install deepspeed

In [ ]:
"""替换自我认知self_cognition数据集中的名称
"""
%cd /content/LLaMA-Factory
!sed -i 's/<NAME>/法律AI/g' data/self_cognition.json
!sed -i 's/<AUTHOR>/billvsme/g' data/self_cognition.json

In [ ]:
"""训练
指令监督微调，lora方式，使用self_cognition数据集

由于没有对话历史，template使用vanilla
"""
%cd /content/LLaMA-Factory
!rm -rf saves/Phi1.5-1.3B/lora/my
!python src/train_bash.py \
    --stage sft \
    --model_name_or_path microsoft/phi-1_5 \
    --do_train True\
    --finetuning_type lora \
    --template vanilla \
    --flash_attn False \
    --shift_attn False \
    --dataset_dir data \
    --dataset self_cognition \
    --cutoff_len 1024 \
    --learning_rate 2e-04 \
    --num_train_epochs 20.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --per_device_eval_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --neft_alpha 0 \
    --train_on_prompt False \
    --upcast_layernorm False \
    --lora_rank 8 \
    --lora_dropout 0.1 \
    --lora_target Wqkv \
    --resume_lora_training True \
    --output_dir saves/Phi1.5-1.3B/lora/my \
    --fp16 True \
    --plot_loss True

In [ ]:
"""导出模型
"""
%cd /content/LLaMA-Factory
!mkdir out_model
!python src/export_model.py \
    --model_name_or_path  microsoft/phi-1_5 \
    --template vanilla \
    --finetuning_type lora \
    --checkpoint_dir saves/Phi1.5-1.3B/lora/my \
    --export_dir out_model/my

In [ ]:
"""查看模型效果
"""

%cd /content/LLaMA-Factory

import os
from threading import Thread

import torch
from transformers.generation.streamers import TextIteratorStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

torch.set_default_device("cuda")

model_name = "/content/LLaMA-Factory/out_model/my/"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


def stream(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, return_attention_mask=True)
    streamer = TextIteratorStreamer(
        tokenizer,
        decode_kwargs={"skip_special_tokens": True})
    Thread(
        target=model.generate, kwargs=dict(
            inputs, streamer=streamer,
            max_new_tokens=128)
    ).start()

    first = True
    for text in streamer:
        if first and text:
            first = False
            continue

        if not text:
            continue
        if tokenizer.eos_token in text:
            break

        yield text


def main():
    welcome_prompt = "欢迎使用 法律AI 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序"
    print(welcome_prompt)
    while True:
        query = input("\n用户：")
        if query.strip() == "stop":
            break
        if query.strip() == "clear":
            os.system("clr")
            print(welcome_prompt)
            continue
        print("\n法律AI：", end="")
        for text in stream(query):
            print(text, end="", flush=True)
        print("")



main()

In [ ]:
"""保存训练好的模型到Google云盘
"""
%cd /content
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir /content/drive/MyDrive/llm_model/
!cp -r /content/LLaMA-Factory/out_model/my /content/drive/MyDrive/llm_model/
